In [1]:
# Setup all functions to run at once and return all the results to user

####################
# Import Packages
####################
import pandas as pd
import sqlite3
import math
import numpy as np

import sys
import os

####################
# Import Functions
####################
from ration_balancer_functions import read_input, fl_get_rows, get_nutrient_intakes


# Modify path temporarily to import functions from NASEM_functions directory
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from NASEM_functions.ration_balancing_equations import calculate_Dt_DMIn_Lact1


In [4]:
def NASEM_model():
########################################
# Step 1: Read User Input
########################################
    # animal_input is a dictionary with all animal specific parameters
    # diet_info is a dataframe with the user entered feed ingredients and %DM intakes
    diet_info, animal_input, equation_selection = read_input('input.txt')
    
    # list_of_feeds is used to query the database and retrieve the ingredient composition, stored in feed_data
    list_of_feeds = diet_info['Feedstuff'].tolist()
    feed_data = fl_get_rows(list_of_feeds)

    # Scale %_DM_intake to 100%
    user_perc = diet_info['%_DM_user'].sum()
    scaling_factor = 100 / user_perc
    diet_info['%_DM_intake'] = diet_info['%_DM_user'] * scaling_factor
    # Adjust so sum is exactly 100
    adjustment = 100 - diet_info['%_DM_intake'].sum()
    diet_info['%_DM_intake'] += adjustment / len(diet_info)

    diet_info['kg_intake'] = diet_info['%_DM_intake'] / 100 * animal_input['DMI']

    # Predict DMI
    if equation_selection['DMI_pred'] == 0:
        animal_input['DMI'] = calculate_Dt_DMIn_Lact1(animal_input['An_Parity_rl'], animal_input['Trg_MilkProd'], animal_input['An_BW'], animal_input['An_BCS'],
                                                      animal_input['An_LactDay'], animal_input['Trg_MilkFatp'], animal_input['Trg_MilkTPp'], animal_input['Trg_MilkLacp'])

########################################
# Step 2: Feed Based Calculations
########################################
    diet_info = get_nutrient_intakes(diet_info, feed_data, animal_input, equation_selection)

    # Return so they can be viewed in environment
    return diet_info, animal_input, feed_data, equation_selection

In [5]:
# Assign values here so they can be see in environment
diet_info, animal_input, feed_data, equation_selection = NASEM_model()